In [ ]:
import requests as r
from bs4 import BeautifulSoup
from collections import Counter
import time
import threading
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed


HOST = 'https://www.104.com.tw'
HEADER = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.96 Safari/537.36'}

# global variable
thread_list=[]
alinks = []
wc = Counter()
lock = threading.Lock()
link_path = 'link_104.csv'

Get all links from 150 pages
for i in range(1,150):
    url =  HOST + '/jobbank/joblist/joblist.cfm?jobsource=n104bank1&ro=1&keyword=%E8%BB%9F%E9%AB%94%E5%B7%A5%E7%A8%8B%E5%B8%AB&order=1&asc=0&page={}'.format(i)
    th = threading.Thread(target=get_pagelinks, args=(url, lock, i, link_path))
    th.start()
    time.sleep(3)
    thread_list.append(th)
else: 
    print('[Debug] All threads are started')

# Join all threads
for thread in thread_list:
    thread.join()

# Get all content from alllinks
# with ThreadPoolExecutor(max_workers=10) as executor:
#     for idx, link in enumerate(alinks):
#         executor.submit(grab_content, link, lock, idx) 
 

with open(link_path, 'r') as f:
    alinks = f.read().strip().split('\n')
    
thread_list2 = []
for idx, link in enumerate(alinks):
    th = threading.Thread(target=grab_content, args=(link, lock, idx ))
    th.start()
    time.sleep(0.5)
    thread_list2.append(th)
for thread in thread_list2:
    thread.join()
        
        
# Wirte results to file
with open('../data_final/104.csv', 'w') as fw2:
    for lang, counts in wc.most_common():
        fw2.write('{},{}\n'.format(lang,counts))

In [1]:
def grab_content(url,lock,idx): #一個用來分析內頁的function
    global HEADER
    
    count = 5
    while count:
        try:
            res2 = r.get(url, headers=HEADER)
            if res2.status_code == 200:
                soup = BeautifulSoup(res2.text, 'lxml')
                lans = soup.select('.tool')[0].select('a')  #lans  is  list
                
                global wc
                with lock:      
                    for lan in lans: 
                        if lan.text in wc:  
                            wc[lan.text] += 1
                        else:
                            wc[lan.text] = 1
                break
            else:
                print ('Cannot retrieve contentfrom page {} Retry: {} '.format(idx, 6 - count))
                count -= 1
                time.sleep(0.5)
                
        except Exception as e:
            print ('Cannot retrieve contentfrom page {} Retry: {} '.format(idx, 6 - count))
            count -= 1
            time.sleep(0.5)
    
    if count == 0:
         print('Retrieve contetn failed page {} '.format(idx))
        


In [2]:
def get_pagelinks(url, lock, i, link_path):  #找所有連結
    
    global HEADER
    global alinks
    global HOST
    
    page_links = [] 
   
    count = 5             # retry 5 times
    while count:
        try:
            res = r.get(url, headers=HEADER)
            if res.status_code == 200 and len(links) != 0:

                soup = BeautifulSoup(res.text, 'lxml')
                links = soup.select(".j_cont > ul > li  > div > a")
                for link in links:
                    if("jobno" in link['href']):  
                         page_links.append(HOST + link['href']) #href put in innerURL function           
                print('Get {} links from page {}'.format(len(page_links), i))

                with open(link_path, 'a') as fw:
                    with lock:
                        total_links += len(page_links)
                        alinks += page_links
                        fw.write('\n'.join(page_links))
                        fw.write('\n')
                    break
            else:
                print ('Cannot retrieve links from page {} Retry: {} '.format(i, 6 - count))
                count -= 1
                time.sleep(1)
        except Exception as e:
            print ('Cannot retrieve links from page {} Retry: {} '.format(i, 6 - count))
            count -= 1
            time.sleep(1)
    if count == 0:
        print('[Error] Page {:2} failed'.format(i))

In [ ]:
# abandoned
def grab_threadpool(alinks, lock):
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(grab_content, link, lock) for link in alinks]
        print('page is finish')

In [ ]:
# abandoned 
class GrabLinkThread (threading.Thread): # new MyClass extends Thread  
    
    def __init__(self, url, lock):   # __init__  is a constructor    #args傳入
        super(GrabLinkThread, self).__init__()  #super()  傳到父類別的建構子
        self.url = url
        self.lock = lock
    
    def run(self):   #等於java中的run  overwrite  
        pageURL(url, self.lock)

In [10]:
wc.most_common()


[('C++', 620),
 ('C', 487),
 ('Java', 461),
 ('C#', 447),
 ('JavaScript', 338),
 ('MS SQL', 302),
 ('Linux', 300),
 ('HTML', 241),
 ('ASP.NET', 218),
 ('jQuery', 186),
 ('MySQL', 179),
 ('PHP', 151),
 ('Android', 147),
 ('Word', 139),
 ('Excel', 137),
 ('Visual C++', 126),
 ('Oracle', 115),
 ('PowerPoint', 101),
 ('Visual Studio .net', 97),
 ('Visual C#', 96),
 ('Objective-C', 96),
 ('Visual Basic .net', 95),
 ('AJAX', 88),
 ('JSP', 87),
 ('Python', 86),
 ('C++.Net', 83),
 ('iOS', 79),
 ('XML', 78),
 ('Visual Studio', 72),
 ('Outlook', 66),
 ('Visual Basic', 63),
 ('TCP/IP', 54),
 ('Windows 7', 53),
 ('Spring', 52),
 ('J2EE', 51),
 ('DHTML', 47),
 ('Windows Server 2008', 43),
 ('HTTP', 42),
 ('MFC', 42),
 ('Windows XP', 40),
 ('OOP', 38),
 ('PL/SQL', 36),
 ('Struts', 35),
 ('Windows Server 2012', 34),
 ('Tomcat', 34),
 ('SWIFT', 34),
 ('Delphi', 32),
 ('Windows 2000', 32),
 ('JDBC', 31),
 ('LabVIEW', 30),
 ('ASP', 29),
 ('Mac OS', 28),
 ('Windows 8', 27),
 ('WinForm', 26),
 ('AutoCAD',